In [ ]:
import h5py

# Abrir train.h5
with h5py.File("../data/train.h5", "r") as f:
    print("Keys disponibles:", list(f.keys()))

    # Si hay un grupo de features, explora dimensiones
    for k in f.keys():
        print(k, f[k].shape, f[k].dtype)


Keys disponibles: ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x', 'body_gyro_y', 'body_gyro_z', 'total_acc_x', 'total_acc_y', 'total_acc_z', 'y']
body_acc_x (7352, 128) float64
body_acc_y (7352, 128) float64
body_acc_z (7352, 128) float64
body_gyro_x (7352, 128) float64
body_gyro_y (7352, 128) float64
body_gyro_z (7352, 128) float64
total_acc_x (7352, 128) float64
total_acc_y (7352, 128) float64
total_acc_z (7352, 128) float64
y (7352,) float64
